<div>
    <span>
    <p align="left">
    <img align="left" style="padding-right: 5px" valign="center" src="https://ga-dash.s3.amazonaws.com/production/assets/logo-9f88ae6c9c3871690e33280fcf557f33.png" width="28px">
    </p>
    </span>
    <span>
        <h1>Joining Table with Pandas</h1>
    </span>
</div>

Pandas provides support for combining `Series`, `DataFrame` and even `xarray` (3 dimensional `DataFrame`s, formerly known in pandas v0.20.0 as `Panel`s) objects with various kinds of set logic for the indicies and relational algebra functionality in the case of join / merge-type operations. More simply stated, this allows you to combine `DataFrame`s.

<!-- Overview -->
<details>
    <summary>Overview</summary>
    <ul>
        <li><b>In this session, we'll cover:</b></li>
        <br>
        <ul>
            <li>Concatenating objects with <code>.append()</code> and <code>.concat()</code></li>
            <li>Combining objects with <code>.join()</code> and <code>.merge()</code></li>
            <li>Combining timeseries objects with <code>.merge_ordered()</code></li>
            <li>Traditionally, this functionality is performed in a relational database, such as <a href="https://pandas.pydata.org/pandas-docs/stable/comparison_with_sql.html#compare-with-sql-join">SQL</a>. With pandas, you'll be able to perform the same operations - in python! The backend is <code>numpy</code>, a powerful linear algebra library which helps keep things speedy</li>
        </ul>
        <br>
        <li><b>Why Join?</b></li>
        <br>
        <ul>
            <li>You might be asking yourself - why keep data separated in different files? <i>Why not just keep it all in one file?</i></li>
            <li>The answer stems from a thing called <a href="https://support.microsoft.com/en-us/help/283878/description-of-the-database-normalization-basics">database normalization</a>. When a database is <i>normalized</i>, it is structured in such a way that redundancy of data is minimized. This allows a database to be faster, smaller, and more flexible when it comes time to change the data inside of it</li>
            <li>The manifestation of this <i>normalization</i> is data that is represented within multiple <a href="https://en.wikipedia.org/wiki/Table_(database)">tables</a> (which are effectively dataframes), related to each other by <a href="https://www.studytonight.com/dbms/database-key.php">keys</a>, or columns in one table that equal a column in another table, allowing them to be joined. In this case, our tables are the <code>.csv</code> files we'll be importing</li>
        </ul>
    </ul>
</details>

<!-- TOC -->
<details>
    <summary>Table of Contents</summary>
    <ul>
        <li><a href="#import">Import</a></li>
        <li><a href="#conapp">Concatenate and Append</a></li>
        <ul>
            <li><a href="#concatenate">Concatenate</a></li>
            <li><a href="#append">Append</a></li>
        </ul>
        <li><a href="#joining">Joining</a></li>
        <ul>
            <li><a href="join">Join</a></li>
            <li><a href="#merge">Merge</a></li>
            <ul>
                <li><a href="#merge_keycols">Merge on Non-Index Columns</a></li>
                <li><a href="#yourturn">Now it's Your Turn!</a></li>
            </ul>
        </ul>
        <li><a href="#exercise">Exercise - AdventureWorks</a></li>
        <ul>
            <li><a href="#p_exercise">Table Joins on Live Data</a></li>
            <ul>
                <li><a href="#ex_pp">Join Product Tables</a></li>
                <li><a href="#ex_soh_sod">Join Sales Order Header and Sales Order Detail Tables</a></li>
                <li><a href="#ex_soh_sod_pt">Join Sales Order Header, Sales Order Detail, and Product Tables</a></li>
            </ul>
        </ul>
    </ul>
</details>

<div id="import"></div>
<h2>Import Pandas</h2>

In [1]:
import pandas as pd
import numpy as np
print(f'Pandas v{pd.__version__}\nNumpy v{np.__version__}')

Pandas v0.23.4
Numpy v1.15.4


<div id="conapp"></div>
<h2>Concatenate and Append</h2>

<div id="concatenate"></div>
<h3>Concatenate</h3>

Concatenate sticks dataframes together, either on top of each other, or next to each other.

```python
Signature: pd.concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False, keys=None, levels=None, names=None, verify_integrity=False, sort=None, copy=True)
Docstring:
Concatenate pandas objects along a particular axis with optional set logic
along the other axes.
```

First, let's create two dataframes, `df1` and `df2`.

In [2]:
# KEEP
df1 = pd.DataFrame([['a', 1], ['b', 2]], columns=['letter', 'number'])
df1.head()

,letter,number
0,a,1
1,b,2


In [3]:
# KEEP
df2 = pd.DataFrame([['c', 3], ['d', 4]], columns=['letter', 'number'])
df2.head()

,letter,number
0,c,3
1,d,4


Next, let's stick the dataframes on top of each other using `concat`. 

In [7]:
pd.concat([df1, df2]).reset_index()

,index,letter,number
0,0,a,1
1,1,b,2
2,0,c,3
3,1,d,4


Finally, let's stick the dataframes <b>next</b> to each other using `concat`. Use of the `axis` kwarg will help us here.

In [8]:
pd.concat([df1, df2], axis=1)

,letter,number,letter,number
0,a,1,c,3
1,b,2,d,4


# <div id="append"></div>
<h3>Append</h3>

Append is very similar to `concat`, except it limits itself to a specific case of `concat`, where `axis=0` (stack on top of each other) and `join=outer` (how to handle the axis of the second dataframe). For almost all cases, `concat` has all the functionality of `append` (and more) and can replace `append` entirely.

```python
Signature: pd.DataFrame.append(self, other, ignore_index=False, verify_integrity=False, sort=None)
Docstring:
Append rows of `other` to the end of this frame, returning a new
object. Columns not in this frame are added as new columns.
```

Also note that `append` is a DataFrame and Series method, and not a pandas library function like `concat` is.

In [9]:
df1.append(df2)

,letter,number
0,a,1
1,b,2
0,c,3
1,d,4


<div id="joining"></div>
<h2>Joining</h2>

<div id="join"></div>
<h3>Join</h3>

`join` allows us to compare two dataframes, and combine them by using a matching column known as a `key`. Normally, during joins, this key is explicitly stated (we'll get to this with `merge` in our next example). With `join`, the `key` joining the table is always the `index` of the first table with (by default) the index of the second table. 

```python
Signature: pd.DataFrame.join(self, other, on=None, how='left', lsuffix='', rsuffix='', sort=False)
Docstring:
Join columns with other DataFrame either on index or on a key
column. Efficiently Join multiple DataFrame objects by index at once by
passing a list.
```

First, let's create two dataframes.

In [13]:
# KEEP
df1 = pd.DataFrame([['a', 1], ['b', 2], ['c', 3], ['d', 4]], columns=['letter', 'number'])
df1.head()

,letter,number
0,a,1
1,b,2
2,c,3
3,d,4


In [14]:
# KEEP
df2 = pd.DataFrame([['e', 5], ['f', 6]], columns=['letter', 'number'])
df2.head()

,letter,number
0,e,5
1,f,6


Now, lets `join` these two dataframes. Note that we will `key`, or 'line up', the two dataframes based on their `indicies`.

Note that, when joining dataframes with any common column names, we will need to supply a `lsuffix` or `rsuffix` kwarg. This is appended to the end of the column name of the returned, joined dataframe to differentiate and identify the source column. Here, we'll use `_df1` to identify that the column shown came from the `df1` dataframe, and `_df2` as a suffix to identify its origin as the `df2` dataframe. 

In [12]:
df1.append(df2)

,letter,number
0,a,1
1,b,2
2,c,3
3,d,4
0,e,5
1,f,6


Note how we have joined the two dataframes on their indicies, which creates a null for rows of index 2 and 3 in `df2`. This is expected and correct.

Also note that the default join behavior of `join` is `left`. We can change this with the `how` kwarg.

For reference, here are the common types of joins. Join types won't be covered in this lesson.
<p align="center">
<img width="500px" src="https://i.stack.imgur.com/udQpD.jpg">
</p>

The type of join we performed above is shown in the upper-left most figure in the above chart.

<div id="merge"></div>
<h3>Merge</h3>

Similar to `join` is `merge`. The difference between the two is the <i>keying behavior</i>. `merge` has a richer API (more functionality) and allows one to join on columns in the source dataframe <i>other than the index</i>. Because `merge` can effectively do everything that `join` can do, and more - it is recommended to always use `merge` unless code brevity is the top concern. 

```python
Signature: pd.merge(left, right, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, sort=False, suffixes=('_x', '_y'), copy=True, indicator=False, validate=None)
Docstring:
Merge DataFrame objects by performing a database-style join operation by
columns or indexes.
```

Note that `merge` is <i>both</i> a DataFrame method as well as a pandas function. Below, we'll be using the pandas function, `pd.merge()`.

In [ ]:
pd.merge(df1, df2

In [16]:
from IPython.display import display
from IPython.display import Image
raw_data = {
        'subject_id': ['1', '2', '3', '4', '5', '6', '7'],
        'first_name': ['Alex', 'Amy', 'Allen', 'Alice', 'Ayoung', 'Joe', 'Mark'],
        'last_name': ['Anderson', 'Ackerman', 'Ali', 'Aoni', 'Atiches', 'Smith', 'Johnson']}
df_a = pd.DataFrame(raw_data, columns = ['subject_id', 'first_name', 'last_name'])
df_a


,subject_id,first_name,last_name
0,1,Alex,Anderson
1,2,Amy,Ackerman
2,3,Allen,Ali
3,4,Alice,Aoni
4,5,Ayoung,Atiches
5,6,Joe,Smith
6,7,Mark,Johnson


In [17]:
raw_data = {
        'subject_id': ['1', '2', '3', '4', '5', '10'],
        'test_score': [91, 85, 95, 71, 86, 90]}
df_n = pd.DataFrame(raw_data, columns = ['subject_id','test_score'])
df_n

,subject_id,test_score
0,1,91
1,2,85
2,3,95
3,4,71
4,5,86
5,10,90


In [18]:
# left join
pd.merge(df_a,df_n,on='subject_id', how='left')

,subject_id,first_name,last_name,test_score
0,1,Alex,Anderson,91.0
1,2,Amy,Ackerman,85.0
2,3,Allen,Ali,95.0
3,4,Alice,Aoni,71.0
4,5,Ayoung,Atiches,86.0
5,6,Joe,Smith,NaN
6,7,Mark,Johnson,NaN


In [20]:
pd.merge(df_a, df_n, left_on='subject_id', right_on = 'subject_id', how='inner')

,subject_id,first_name,last_name,test_score
0,1,Alex,Anderson,91
1,2,Amy,Ackerman,85
2,3,Allen,Ali,95
3,4,Alice,Aoni,71
4,5,Ayoung,Atiches,86


In [21]:
raw_data = {
        'customer_id': ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10'],
        'name': ['Bob', 'Susan', 'Ross', 'Amy', 'Molly', 'Ellie', 'Joe', 'Matt', 'Natalie', 'Erica']}
df1 = pd.DataFrame(raw_data, columns = ['customer_id', 'name'])
df1


,customer_id,name
0,1,Bob
1,2,Susan
2,3,Ross
3,4,Amy
4,5,Molly
5,6,Ellie
6,7,Joe
7,8,Matt
8,9,Natalie
9,10,Erica


In [22]:
raw_data = {
        'customer_id': ['1', '2', '4', '6', '7', '8'],
        'product': ['Samsung Phone', 'iPhone', 'iPhone', 'iPad', 'MacBook Pro', 'Galaxy Phone']}
df2 = pd.DataFrame(raw_data, columns = ['customer_id', 'product'])
df2

,customer_id,product
0,1,Samsung Phone
1,2,iPhone
2,4,iPhone
3,6,iPad
4,7,MacBook Pro
5,8,Galaxy Phone


In [31]:
pd.merge(df1,df2,left_on ='customer_id',right_on='customer_id', how ='inner')

,customer_id,name,product
0,1,Bob,Samsung Phone
1,2,Susan,iPhone
2,4,Amy,iPhone
3,6,Ellie,iPad
4,7,Joe,MacBook Pro
5,8,Matt,Galaxy Phone


Note that we've achieved the same exact output as we did with `join`, but it took a little more explicit work. Let's run through the arguments for clarity:

<ul>
    <li><code>df1</code>: this is the first dataframe, and considered to be on the 'left' of <code>df2</code></li>
    <li><code>df2</code>: this is the second dataframe, considered to be on the right of <code>df1</code></li>
    <li><code>how='left'</code>: this states the type of join; see the above SQL join table</li>
    <li><code>left_index=True</code>: this uses the index of <code>df1</code> as the join key for the left table</li>
    <li><code>right_index=True</code>: this uses the index of <code>df2</code> as the join key for the right table</li>
    <li><code>suffixes</code>: this places <code>_df1</code> after column names which came from <code>df1</code></li>
</ul>

<div id="merge_keycols"></div>
<h4>Merge on Non-Index Columns</h4>

This brings us to our next point: merging on columns that are not the index columns. This is very, very common in SQL joins and this technique can be used to join just about any DataFrame.

First, let's create some more realistic data - stocks!

In [32]:
# KEEP
openprice = pd.DataFrame({'Symbol': ['AAPL', 'DHR', 'DAL', 'AMZN'], 'OpenPrice': [217.51, 96.54, 51.45, 1703.34]})
wkhigh = pd.DataFrame({'Symbol': ['DAL', 'AMZN', 'AAPL', 'DHR'], '52wkHigh': [60.79, 2050.49, 233.47, 110.11]})
stockname = pd.DataFrame({'Symbol': ['AMZN', 'DHR', 'DAL', 'AAPL'], 'Name': ['Amazon', 'Danaher', 'Delta Airlines', 'Apple']})

Now, let's join the <code>openprice</code> and <code>wkhigh</code> dataframes together.

In [34]:
pd.merge(openprice,wkhigh, left_on = 'Symbol', right_on ='Symbol', how = 'inner')

,Symbol,OpenPrice,52wkHigh
0,AAPL,217.51,233.47
1,DHR,96.54,110.11
2,DAL,51.45,60.79
3,AMZN,1703.34,2050.49


Note how our `Symbol` column isn't in the same order in each dataframe. This is intentional, and note that the dataframe on the left, `openprice` dictates the order of the dataframe on the right, `wkhigh`. Also note that the shared key between the two dataframes is exempt from having a <code>suffix</code> applied to it. 

<div id="yourturn"></div>
<h4>Now it's your turn!</h4>

<ul>
    <li><code>merge</code> the <code>openprice</code> and <code>stockname</code> dataframes and inspect the result</li>
    <li><code>merge</code> all three dataframes together and inspect the result</li>
</ul>

<div id="exercise"></div>
<h2>Exercise - Adventure Works</h2>
<p align="right">
<img src="http://lh6.ggpht.com/_XjcDyZkJqHg/TPaaRcaysbI/AAAAAAAAAFo/b1U3q-qbTjY/AdventureWorks%20Logo%5B5%5D.png?imgmax=800">
</p>

<div id="p_exercise"></div>
<h3>Table Joins on Live Data</h3>

Here are the data dictionaries we'll be using for the following exercise:

<ul>
    <li><a href="https://www.sqldatadictionary.com/AdventureWorks2014/Production.Product.html">Production.Product</a></li>
    <li><a href="https://www.sqldatadictionary.com/AdventureWorks2014/Production.ProductSubCategory.html">Production.ProductSubcategory</a></li>
    <li><a href="https://www.sqldatadictionary.com/AdventureWorks2014/Sales.SalesOrderHeader.html">Sales.SalesOrderHeader</a></li>
    <li><a href="https://www.sqldatadictionary.com/AdventureWorks2014/Sales.SalesOrderDetail.html">Sales.SalesOrderDetail</a></li>
</ul>

In [35]:
p = pd.read_csv('../data/Production.Product.csv', sep='\t')
ps = pd.read_csv('../data/Production.ProductSubcategory.csv', sep='\t')
soh = pd.read_csv('../data/Sales.SalesOrderHeader.csv', sep='\t', nrows=1000)
sod = pd.read_csv('../data/Sales.SalesOrderDetail.csv', sep='\t', nrows=1000)

<div id="ex_pp"></div>
<h4>Join Product Tables</h4>

<ul>
    <li>Using the <code>Production.Product.ProductID</code> and <code>Production.ProductSubcategory.ProductID</code> keys, join the <code>Production.Product</code> and <code>Production.ProductSubcategory</code> tables</li>
</ul>

<div id="ex_soh_sod"></div>
<h4>Join Sales Order Header and Sales Order Detail Tables</h4>

<ul>
    <li>Join the <code>Sales.SalesOrderHeader</code> and <code>Sales.SalesOrderDetail</code> tables</li>
    <li>Don't forget to use your data dictionaries!</li>
</ul>

<div id="ex_soh_sod_pt"></div>
<h4>Join Sales Order Header, Sales Order Detail, and Product Tables</h4>

<ul>
    <li>Join the <code>Sales.SalesOrderHeader</code>, <code>Sales.SalesOrderDetail</code>, and <code>Production.Product</code> tables</li>
    <li>Don't forget to use your data dictionaries!</li>
</ul>